In [1]:
import pandas as pd
df= pd.read_csv('boundary_corelation.csv')## import the correlation matrix
from sklearn.cluster import OPTICS
import numpy as np
import time

In [2]:
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,0.000,0.835,0.585,0.416,1.000,0.936,1.000,0.939,0.413,1.000,...,0.763,0.388,0.546,1.000,0.847,0.941,1.000,0.183,0.374,0.330
1,0.835,0.000,0.411,0.622,0.338,0.151,0.357,0.190,0.748,1.000,...,0.867,0.868,0.902,0.368,0.136,0.214,0.694,0.865,0.865,0.855
2,0.585,0.411,0.000,0.372,0.589,0.506,0.601,0.529,0.497,1.000,...,0.758,0.669,0.754,0.608,0.457,0.512,0.580,0.661,0.661,0.637
3,0.416,0.622,0.372,0.000,0.793,0.719,0.798,0.732,0.182,1.000,...,0.762,0.534,0.654,0.802,0.651,0.740,0.846,0.523,0.522,0.489
4,1.000,0.338,0.589,0.793,0.000,0.325,0.028,0.311,0.917,0.845,...,1.000,1.000,1.000,0.045,0.390,0.331,0.701,1.000,0.963,1.000


In [3]:
df_new=pd.DataFrame()
df_new['pathid']=df.index+1
df_new['pathid']=df_new['pathid'].replace([63],['Clusters_found'])
df_new = df_new.append({'pathid':'Noise_count'},ignore_index=True)


In [4]:
df_new.tail(5)

,pathid
59,60
60,61
61,62
62,Clusters_found
63,Noise_count


In [ ]:
# epsilons= [0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55]##if cluster_method set to 'xi', then the epsilons needed to use default
# epsilons= [0.2,0.25,0.3,0.35,0.4,0.45]
min_support=[2,3,4]
print("############# OPTICS #############")
start_time = time.time()
# for eps in epsilons:
for mins in min_support:
    df= pd.read_csv('boundary_corelation.csv')
#drop the index 62 for both column and row as the last path is not needed
    df=df.drop('62',axis=1,inplace=False)
    df=df.drop(62)
##run OPTICS   -- cluster_method = 'xi' is default, however, the result of dbscan seems better as it reduces the number of clusters 
    db= OPTICS(min_samples=mins, metric="precomputed").fit(df)##max_eps=0.2,default is 0.55
    print(db)
#     print(db.get_params(deep=True))
    labels=db.labels_
    labelx=labels
##print the result of OPTICS 
    print(labels[0::])
    print("===============================")
    ##print the total number of clusters (including noise : -1)
    cluster=np.unique(labels)
    num_cluster=len(cluster)-1
    modified_clusters = np.delete(labels, np.where(labels == -1))
    frequency=np.array(np.unique(modified_clusters, return_counts=True)).T
    print('Identified clusters excluding noise:',num_cluster,"=>\n",frequency)

#     name= 'label_'+ str(eps)+'_'+str(mins)
    name= 'label_'+'default'+'_'+str(mins)
    ##counting the noise number in each set
    noise= list(labels).count(-1)
    labels=np.append(labels,num_cluster)
    labels=np.append(labels,noise)
    df_new[name]=labels
    print("Identifide noise",noise)
    print("===============================\n")
end_time = time.time()
print("time take =", end_time - start_time)

In [ ]:
df_new.tail(5)

In [ ]:
df_new.to_csv('total_OPTICS_default_output.csv')

### for generating the csv file of OPTICS "default" 

In [10]:
########### for generating the csv file of optics default #####
df_updated= pd.read_csv('boundary_path_cts.csv')
name_contains = 'label_'
drop_lst = [col for col in (df_updated.columns) if col.startswith(name_contains)]
df_updated = df_updated.drop(drop_lst, axis=1)
df_updated.head(2)
## ========== Default ==========with only one param: min_support and the method used is Xi
min_support=[2,3,4]
start_time = time.time()
for mins in min_support:
    df= pd.read_csv('boundary_corelation.csv')
    df=df.drop('62',axis=1,inplace=False)
    df=df.drop(62)
    ##run OPTICS
    db= OPTICS(min_samples=mins, metric="precomputed").fit(df)
    labels=db.labels_
    labelx=labels

    ##create the column name with the minsuppor used ex: label_default_2 (Minsupport:2)
    name= 'label_'+'default'+'_'+str(mins)
    ##counting the noise number in each set
    df_updated[name]=labels
end_time = time.time()
print("time take =", end_time - start_time)
df_updated
df_updated.to_csv('first_step_OPTICS_default.csv')##this is the output from OPTICS combined with path frequency

time take = 0.06681108474731445
